Import Dependencies

In [11]:
import time
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
import re 
import pandas as pd

Helper function to get chrome driver initiated

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)
    

Get Mars news_data 

In [55]:
browser = init_browser()
# create surf_data dict that we can insert into mongo
news_data = {}

# visit mars.nasa.com
nasa = "https://mars.nasa.gov/news/"
browser.visit(nasa)
time.sleep(2)
html = browser.html

# create a soup object from the html
news = BeautifulSoup(html, "html.parser")

#get news title,text,and date as values in the news_data dict
news_data['title']=news.find("div", class_="content_title").text
print("debug title is {}\n".format(news_data['title']))

news_data['content']=news.find("div", class_="article_teaser_body").text
print("debug content is {}\n".format(news_data['content']))

news_data['time']=news.find("div",class_="list_date").text
print("debug title is {}\n".format(news_data['time']))



debug title is Bound for Mars: Countdown to First Interplanetary Launch from California

debug content is On May 5, millions of Californians may witness the historic first interplanetary launch from America’s West Coast.

debug title is April  6, 2018



Get JPL Mars Space Images - Featured Image

In [69]:
#get link and visit
picture_link="https://www.jpl.nasa.gov/spaceimages/?search=Mars&category=Featured"
browser.visit(picture_link)

# The website has the featured_image with a button with the id "full_image" and click it 
button = browser.find_by_id("full_image")
button.click()
time.sleep(2)
html_pic = browser.html
#pass it to another beautiful soup object
image= BeautifulSoup(html_pic, "html.parser")

#find the image src of the image
#here the image src is just mediumsize
image_src=image.find('img',class_='fancybox-image')['src']
print("debug here I am the src of the featured image {}".format(image_src))

#I exhausted that webpage but really cannot find the large size image of the current image
#I can find the next largesize image but not the current one
#So I decided to extract the image ID and plugged it in the largesize image link

image_id=re.findall('/[a-zA-Z0-9]+_',image_src)[0][1:-1]
fullsize_link='https://www.jpl.nasa.gov/spaceimages/images/largesize/{}_hires.jpg'.format(image_id)
print('hey I am the fullsize image link {}'.format(fullsize_link))
news_data['featured_image']=fullsize_link

debug here I am the src of the featured image /spaceimages/images/mediumsize/PIA18897_ip.jpg
hey I am the fullsize image link https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18897_hires.jpg


Get Mars Weather

In [10]:
browser = init_browser()
weather_link='https://twitter.com/marswxreport?lang=en'
browser.visit(weather_link)
html_weather=browser.html
weather=BeautifulSoup(html_weather,'html.parser')
current_weather=weather.find('div',class_='js-tweet-text-container').p.text
print('debug current weather in Marse is {}'.format(current_weather))
news_data['weather']=current_weather

debug current weather in Marse is Sol 2022 (April 14, 2018), Sunny, high -4C/24F, low -73C/-99F, pressure at 7.19 hPa, daylight 05:27-17:21


Get Marse facts

In [20]:
facts_url='https://space-facts.com/mars/'
facts_table=pd.read_html(facts_url)[0]
facts_table.columns=['Description','Value']
facts_table=facts_table.set_index('Description')
#put it in the dict
news_data['facts_table']=facts_table

In [49]:
facts_table.columns

Index(['Value'], dtype='object')

Get Mars Hemispheres images


In [48]:
browser = init_browser()
hemi_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_image_urls=[]
browser.visit(hemi_url)

# The website has the featured_image with a button with the id "full_image" and click it 
each_hemi_url = browser.find_by_css('div[class="description"] a')
href_list=[i['href'] for i in each_hemi_url ]

#enter for look to visit all the links to the final full image site
for i in href_list:
    browser.visit(i)  #visit the full image url
    html=browser.html  #connect
    #pass it to a beautiful soup object
    final_html=BeautifulSoup(html,'html.parser')
    #get the relative path
    image_relative_path=final_html.find('img',class_='wide-image')['src']
    print("debug hey my relative path is {}".format(image_relative_path))
    #get full link of the image
    image_full_link='https://astrogeology.usgs.gov{}'.format(image_relative_path)
    print('debug hey my full path is {}'.format(image_full_link))
    #get image title
    image_title=final_html.find('h2',class_='title').text
    print('debug hey my title is {}'.format(image_title))
    hemi_image_urls.append({"title":image_title,"image_url":image_full_link})
    print('hey so far this is my list {}'.format(hemi_image_urls))
    time.sleep(2)   # sleep
    
#put it in the news_data dict
news_data['hemi_info']=hemi_image_urls

debug hey my relative path is /cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
debug hey my full path is https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
debug hey my title is Cerberus Hemisphere Enhanced
hey so far this is my list [{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}]
debug hey my relative path is /cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
debug hey my full path is https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
debug hey my title is Schiaparelli Hemisphere Enhanced
hey so far this is my list [{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schia

In [50]:
dic={'a':1,"b":2}

In [51]:
for key,value in dic.items():
    print(key,value)

a 1
b 2
